In [3]:
#//----------------------------
#//LIBRARIES
import math
    #Numeric Python
import numpy as np
    #Pandas (dataframes)
import pandas as pd
    #datetime for fate manipulation
from datetime import date, datetime, timedelta  
    #Regex for advanced string matching
import re
    #for time related stuff
import time
    #json library
import json
    #Analyst tools
import sys
sys.path.append('../')

import matplotlib.pyplot as plt
import seaborn as sns
import altair as alt
import plotly.express as px

In [40]:
#//----------------GET INFO OF SD-------------------------//
sd_cat = pd.read_csv("sd_cat.csv")
sd_subcat = pd.read_csv("sd_subcat.csv")
users_ms = pd.read_csv("MTD_-_CONVERSION_OF_MS_MS_2022_07_25.csv")

In [41]:
sd_subcat = sd_subcat[['category::multi-filter', 'PEN_users ALL', '%_users SDMS/ALL', 'ciudad','microsegment','Contribution ARPU by SDMS', 'subcategoria',
                       'SPEND SDMS','SDMS_DISCOUNT', 'AOV SDMS','SDMS_GMV', 'mes', 'ARPU SDMS','SDMS DISCOUNT_OVER_GMV','Users ALL','week::multi-filter']]

sd_subcat['Users SDMS'] = (sd_subcat['Users ALL']/(sd_subcat['PEN_users ALL']/100))*(sd_subcat['%_users SDMS/ALL']/100)
sd_subcat['gasto']= sd_subcat['SDMS_DISCOUNT']/sd_subcat['Users SDMS']

sd_subcat=sd_subcat.merge(users_ms, how = 'left', left_on=['microsegment', 'ciudad'], right_on=['ms', 'city::multi-filter'])
sd_subcat['CONVERSION'] = sd_subcat['Users SDMS']/sd_subcat['all_customers']

sd_subcat.rename(columns = {'week::multi-filter':'week', 'category::multi-filter':'category'}, inplace = True)

sd_subcat['%_users SDMS/ALL'] = pd.Series([round(x,4) for x in sd_subcat['%_users SDMS/ALL']])
sd_subcat['Contribution ARPU by SDMS'] = pd.Series([round(x,4) for x in sd_subcat['Contribution ARPU by SDMS']])
sd_subcat['gasto'] = pd.Series([round(x,4) for x in sd_subcat['gasto']])
sd_subcat['CONVERSION'] = pd.Series([round(x,4) for x in sd_subcat['CONVERSION']])

In [42]:
sd_subcat.head()

,category,PEN_users ALL,%_users SDMS/ALL,ciudad,microsegment,Contribution ARPU by SDMS,subcategoria,SPEND SDMS,SDMS_DISCOUNT,AOV SDMS,...,Users ALL,week,Users SDMS,gasto,city::multi-filter,ms,all_customers,all_customers_ms,per_conversion,CONVERSION
0,Descartáveis,2.511628,0.0000,VCP,NaN,0.0000,Utensílios,NaN,NaN,NaN,...,27,28,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Lácteos & Huevos,4.155340,0.5437,GDL,other,8.5907,Cremas,13.763605,15.41219,13.763605,...,107,29,14.0,1.1009,GDL,other,5391.0,2293.0,42.533853,0.0026
2,Lácteos & Huevos,1.173021,0.1466,GDL,other,17.9132,Mantequillas,12.225117,0.97759,12.225117,...,8,26,1.0,0.9776,GDL,other,5391.0,2293.0,42.533853,0.0002
3,Lácteos & Huevos,13.830614,2.7195,GDL,other,3.7015,Leches,1.957353,9.26524,1.957353,...,356,27,70.0,0.1324,GDL,other,5391.0,2293.0,42.533853,0.0130
4,Aseo e Higiene,0.040486,0.0000,PBC,NaN,0.0000,Desinfectantes & Sanitizantes,NaN,NaN,NaN,...,1,28,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [53]:
city = 'SPO'
sd_subcat_city = sd_subcat[(sd_subcat['ciudad'] == city)].dropna().copy()

In [54]:
contibution_ARPU = (
    alt.Chart(sd_subcat_city)
    .mark_bar()
    .encode(
        alt.Y(field='Contribution ARPU by SDMS', aggregate='mean', type='quantitative'),
        alt.X('subcategoria', sort='-y', axis=alt.Axis(labelAngle=-45)),
    #column = 'category::multi-filter'
    #color='week::multi-filter'
    )
).properties(title='Contribution ARPU by ALL MS for each SUBCAT')
contibution_ARPU


alt.Chart(...)

In [1]:
# # dropdown filter
# microsegment = sd_subcat_city["microsegment"].unique()
# microsegment_dropdown = alt.binding_select(options=microsegment)
# microsegment_select = alt.selection_single(
#     fields=["microsegment"],
#     bind=microsegment_dropdown,
#     name="Microsegment",
#     empty="all"
# )

microsegment = alt.selection_multi(fields=['microsegment'], bind='legend')

users_Penetration = (
    alt.Chart(sd_subcat_city)
    .mark_bar(color="#FFAA20")
    .encode(
        alt.X('subcategoria', sort='-y'),
        y='%_users SDMS/ALL',
        color='microsegment',
        tooltip=['microsegment:N', 'category:N', 'week:N', '%_users SDMS/ALL:Q']
    #column = 'category::multi-filter'
    #color='week::multi-filter'
    )
    .add_selection(
    microsegment
    )
    .transform_filter(
    microsegment
    )
    # .configure_range(
    # category={'scheme': 'dark2'}
    # )
).properties(title='Penetration of Users by ALL MS for each SUBCAT')
#users_Penetration

NameError: name 'alt' is not defined

In [56]:
selection = alt.selection_multi(fields=['microsegment'], bind='legend')

Gasto_Users = (
    alt.Chart(sd_subcat_city)
    .mark_bar(color="#FFAA20")
    .encode(
        alt.X('subcategoria', sort='-y'),
        y='gasto',
        color='microsegment',
        tooltip=['microsegment:N', 'category:N', 'week:N', 'gasto:Q']
    #column = 'category::multi-filter'
    #color='week::multi-filter'
    )
    .add_selection(
    selection
    )
    .transform_filter(
    selection
    )
    # .configure_range(
    # category={'scheme': 'dark2'}
    # )
).properties(title='Cost customer (Discount_Spend/Pen_Users) by MS for each SUBCAT')
#Gasto_Users

In [57]:
selection = alt.selection_multi(fields=['microsegment'], bind='legend')

Conversion_MS = (
    alt.Chart(sd_subcat_city)
    .mark_bar(color="#FFAA20")
    .encode(
        alt.X('subcategoria', sort='-y'),
        y='CONVERSION',
        color='microsegment',
        tooltip=['microsegment:N', 'category:N', 'week:N', 'CONVERSION:Q']
    #column = 'category::multi-filter'
    #color='week::multi-filter'
    )
    .add_selection(
    selection
    )
    .transform_filter(
    selection
    )
    # .configure_range(
    # category={'scheme': 'dark2'}
    # )
).properties(title='CONVERSION MS (Buyers MS/All Users MS) by SUBCAT')
#Conversion_MS

In [58]:
selection = alt.selection_multi(fields=['microsegment'], bind='legend')

BURN_MS = (
    alt.Chart(sd_subcat_city)
    .mark_bar(color="#FFAA20")
    .encode(
        alt.X('subcategoria', sort='-y'),
        y='SDMS DISCOUNT_OVER_GMV',
        color='microsegment',
        tooltip=['microsegment:N', 'category:N', 'week:N', 'SDMS DISCOUNT_OVER_GMV:Q']
    #column = 'category::multi-filter'
    #color='week::multi-filter'
    )
    .add_selection(
    selection
    )
    .transform_filter(
    selection
    )
    # .configure_range(
    # category={'scheme': 'dark2'}
    # )
).properties(title='BURN by SUBCAT')
#BURN_MS


In [59]:
alt.vconcat(
    alt.hconcat(
        users_Penetration, Gasto_Users)
    .properties(
        title="ALL INFO SUBCATS"
    ),
    alt.hconcat(
        Conversion_MS, BURN_MS)
    .properties(
        #title="Click and drag the bottom bars to filter the scatter plot"    
    )
)

alt.VConcatChart(...)

In [50]:
AOV_Gasto = (
    alt.Chart(sd_subcat_city)
    .mark_point(color='red')
    .encode(
        alt.X(field='gasto', aggregate='mean', type='quantitative', sort='-x'),
        y='AOV SDMS',
        color='microsegment',
        tooltip=['microsegment:N', 'subcategoria:N', 'week:N', 'AOV SDMS:Q', 'gasto:Q']
    #column = 'category::multi-filter'
    #color='week::multi-filter'
    )
    .add_selection(
    selection
    )
    .transform_filter(
    selection
    )
    # .configure_range(
    # category={'scheme': 'dark2'}
    # )
).properties(title='AOV vs Gasto', width=850)
AOV_Gasto

alt.Chart(...)

In [11]:
"""
cities = {}
for i in parametros:
    aux = []
    for j in parametros[i]:
        aux.append(sorted(parametros[i][j]))
    cities[i] = pd.DataFrame(aux).T
    cities[i].columns = list(parametros[i].keys())
    #cities[i].to_csv(f'{i}_MS_SUBCAT.csv', index = False)
"""

"\ncities = {}\nfor i in parametros:\n    aux = []\n    for j in parametros[i]:\n        aux.append(sorted(parametros[i][j]))\n    cities[i] = pd.DataFrame(aux).T\n    cities[i].columns = list(parametros[i].keys())\n    #cities[i].to_csv(f'{i}_MS_SUBCAT.csv', index = False)\n"